In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import warnings
import networkx as nx
import matplotlib.colors as mcolors
# warnings.simplefilter(action='ignore', category=FutureWarning)
# pd.set_option('display.max_rows', None)  # To display all rows
# pd.set_option('display.max_columns', None)  # To display all columns
import glob as glob
import math

# This script is to have create new dataframes with 44 matching harmonized bilateral region suvrs for both adni and a4

* Note in the path for calling the data, replace the /home/mlauber with home/yourfolder

- merged_adni_at_amy_pos.csv should now have "HIPPOCAMPUS_SUVR" with bilateral hippocampus 

#### Both ADNI and A4 used the free surfer recon all command that uses the desikan killany atlas (cerebral cortex into 34 regions)
##### To do: 
- make a list of predefined regions I want to look at and then just harmonize the regional names across the 2 data frames 
- looks like desikan killany atlas is 34 cortical ones but also need to check ones like brainstem etc 

#### Further cleaning: goal is to end up with a4 and adni amy_tau merged csvs with similar naming conventions for each region/col
##### The a4 data has columns with Volume_ prefix so need to drop those 
##### need to makesure that the atlas used between the 2 datasets was consistsent 
- there's also a mean unknown col need to see if that's also present in the adni one 
- also with left right from stanford coumentation "We additionally created bilateral regions taking a volume weighted average across hemispheres. These are provided for in the bi_* columns" so maybe just use these
- need to drop stuff like Mean_non_WM_hypointensities


* additional note- see if the atlas used was consistent between A4 and adni because the naming conventions are different 
* ex mean_ctx_rh_posteriorcingulate in the TAUSUVR vs CTX_RH_PRECENTRAL_SUVR in the UCBERKLEYAV1451_PVC_8mm csv 

In [7]:
#load and analyze amyloid positive a4 data 
merged_a4_at_amypos = pd.read_csv("/home/mlauber/mci_mri_graph/data_paths_and_cleaning/data/intermediate_data/a4/merged_a4_at_amy_pos.csv")

In [4]:
print(merged_a4_at_amypos)

           BID         update_stamp_x       ligand  centiloid  \
0    B10423472  2018-12-06 00:00:00.0  Florbetapir    80.8687   
1    B10693248  2018-12-06 00:00:00.0  Florbetapir   113.8210   
2    B10709603  2018-12-06 00:00:00.0  Florbetapir    40.5933   
3    B10904689  2018-12-06 00:00:00.0  Florbetapir   126.6360   
4    B11089276  2018-12-06 00:00:00.0  Florbetapir    35.1012   
..         ...                    ...          ...        ...   
368  B92372599  2019-04-19 00:00:00.0  Florbetapir    40.5933   
369  B97841288  2019-04-19 00:00:00.0  Florbetapir    71.7152   
370  B98032541  2019-04-19 00:00:00.0  Florbetapir    55.2389   
371  B99033399  2019-04-19 00:00:00.0  Florbetapir    53.4082   
372  B99860801  2019-04-19 00:00:00.0  Florbetapir    35.1012   

     Mean_3rd_Ventricle  Mean_4th_Ventricle  Mean_Brain_Stem  \
0              0.845577            0.864285         0.911925   
1              1.110908            0.963089         1.000662   
2              1.041388    

#### notes from a4 doumentation (stanford pipeline)
- created using freesurfer recon- all 
- seems like a4 ones all start with Mean_

In [231]:
#check how many cols have bi_ prefix 
bi_reg_count = sum(col.startswith('bi_') for col in merged_a4_at_amypos.columns)
print(bi_reg_count)
#50 bilateral regions 

50


In [232]:
filtered_columns = [col for col in merged_a4_at_amypos.columns if col.startswith('bi_')]
column_list = list(filtered_columns)

# Create a DataFrame from the column_list
a4_col_list = pd.DataFrame({'Column Names': column_list})

# Save the DataFrame to a CSV file
a4_col_list.to_csv("./a4/a4_bi_columns.csv", index=False)

In [5]:
#load and analyze adni data
merged_adni_at_amypos = pd.read_csv("/home/mlauber/mci_mri_graph/data_paths_and_cleaning/data/intermediate_data/adni/merged_adni_at_amy_pos.csv")

In [234]:
print(merged_adni_at_amypos['HIPPOCAMPUS_SUVR']) #hippocampus now exists

0      1.596
1      1.562
2      1.169
3      1.145
4      1.328
5      1.772
6      1.709
7      1.364
8      1.223
9      1.229
10     1.197
11     1.040
12     1.325
13     1.655
14     1.423
15     1.184
16     1.196
17     1.091
18     1.143
19     1.991
20     1.030
21     1.176
22     1.216
23     1.241
24     1.823
25     1.473
26     1.418
27     1.458
28     1.461
29     1.106
30     1.195
31     1.422
32     1.197
33     2.505
34     1.799
35     1.441
36     1.789
37     1.445
38     1.209
39     1.359
40     0.943
41     0.963
42     1.366
43     1.129
44     1.070
45     1.829
46     1.229
47     1.715
48     0.923
49     1.105
50     1.098
51     1.830
52     1.417
53     1.539
54     1.222
55     1.285
56     0.944
57     1.375
58     1.548
59     1.136
60     1.445
61     2.091
62     1.585
63     0.984
64     1.054
65     1.277
66     1.034
67     1.204
68     1.325
69     1.175
70     1.372
71     1.632
72     1.245
73     1.554
74     2.675
75     1.746
76     1.206

In [235]:
#seems like the volume columns are named like ******_VOLUME so drop these 
#something like endswith "_VOLUME".drop()
drop_cols = [col for col in merged_adni_at_amypos.columns if col.endswith('_VOLUME')]
merged_adni_at_amypos = merged_adni_at_amypos.drop(columns=drop_cols)

In [236]:
#also drop choroid 
#drop adni startswith("CHOROID")
adni_drop_cols = [col for col in merged_adni_at_amypos.columns if col.startswith('CHOROID')]
merged_adni_at_amypos = merged_adni_at_amypos.drop(columns=adni_drop_cols)

In [237]:
#this is without the volume 
print(merged_adni_at_amypos.columns)

Index(['RID', 'TRACER_x', 'CENTILOIDS', 'LONIUID', 'SITEID', 'PTID',
       'SCANDATE', 'PROCESSDATE', 'TRACER_y', 'META_TEMPORAL_SUVR',
       ...
       'RIGHT_CEREBELLUM_CORTEX_SUVR', 'RIGHT_CEREBELLUM_WHITE_MATTER_SUVR',
       'RIGHT_CEREBRAL_WHITE_MATTER_SUVR', 'RIGHT_CHOROID_PLEXUS_SUVR',
       'RIGHT_HIPPOCAMPUS_SUVR', 'RIGHT_PALLIDUM_SUVR', 'RIGHT_PUTAMEN_SUVR',
       'RIGHT_THALAMUS_PROPER_SUVR', 'RIGHT_VENTRALDC_SUVR', 'update_stamp'],
      dtype='object', length=158)


#### further cleaning adni data (only bilateral (l/r hemi regions combined))
- has col names _VOLUME for the volume so drop these 
- they also have _SUVR so maybe could only keep these as a quality control check 
- the naming conventions are stuff like CTX_LH_SUPERIORFRONTAL_SUVR (there's a CTX_LH_SUPERIORFRONTAL_SUVR and CTX_RH_SUPERIORFRONTAL_SUVR for each) 
- looks like there's a CTX_SUPERIORFRONTAL_SUVR and then a CTX_LH_SUPERIORFRONTAL_SUVR and a CTX_RH_SUPERIORFRONTAL_SUVR
- so need to only keep the CTX_SUPERIORFRONTAL_SUVR ones (something like search start with CTX**) if only looking at bilateral brain regions (this correponds to the A4 naming convention "bi_* columns" )
- drop stuff like TRACER_x and Tracer_y, LONIID


### drop volume before doing anything else

#### Things to drop from A4
- everything starting with Volume 
- ending with 'Ventricle" 
- specific ones ending in _plexus, vessel, _Chiasm, Unknown, _hypointensities, WM, 
- after doing this its 151 rows 

In [238]:
suffixes_to_drop = ['_Ventricle', '_plexus','_vessel','_Chiasm','_hypointensities','WM','CSF','Unknown','_y','_x',"Matter","Vent"]

# Drop columns ending with the specified suffixes
for suffix in suffixes_to_drop:
    a4_drop_cols = [col for col in merged_a4_at_amypos.columns if col.endswith(suffix)]
    merged_a4_at_amypos = merged_a4_at_amypos.drop(columns=a4_drop_cols)

In [239]:
#seems like the volume columns are named like Volume_**** so drop these columns 
#something like starts with "Volume_***".drop()
a4_drop_cols = [col for col in merged_a4_at_amypos.columns if col.startswith('Volume_')]
merged_a4_at_amypos = merged_a4_at_amypos.drop(columns=a4_drop_cols)



In [240]:
merged_a4_at_amypos.head(5)

,BID,ligand,centiloid,Mean_Brain_Stem,Mean_CC_Anterior,Mean_CC_Central,Mean_CC_Mid_Anterior,Mean_CC_Mid_Posterior,Mean_CC_Posterior,Mean_Left_Accumbens_area,Mean_Left_Amygdala,Mean_Left_Caudate,Mean_Left_Cerebellum_Cortex,Mean_Left_Hippocampus,Mean_Left_Pallidum,Mean_Left_Putamen,Mean_Left_Thalamus,Mean_Left_VentralDC,Mean_Right_Accumbens_area,Mean_Right_Amygdala,Mean_Right_Caudate,Mean_Right_Cerebellum_Cortex,Mean_Right_Hippocampus,Mean_Right_Pallidum,Mean_Right_Putamen,Mean_Right_Thalamus,Mean_Right_VentralDC,Mean_ctx_lh_bankssts,Mean_ctx_lh_caudalanteriorcingulate,Mean_ctx_lh_caudalmiddlefrontal,Mean_ctx_lh_cuneus,Mean_ctx_lh_entorhinal,Mean_ctx_lh_frontalpole,Mean_ctx_lh_fusiform,Mean_ctx_lh_inferiorparietal,Mean_ctx_lh_inferiortemporal,Mean_ctx_lh_insula,Mean_ctx_lh_isthmuscingulate,Mean_ctx_lh_lateraloccipital,Mean_ctx_lh_lateralorbitofrontal,Mean_ctx_lh_lingual,Mean_ctx_lh_medialorbitofrontal,Mean_ctx_lh_middletemporal,Mean_ctx_lh_paracentral,Mean_ctx_lh_parahippocampal,Mean_ctx_lh_parsopercularis,Mean_ctx_lh_parsorbitalis,Mean_ctx_lh_parstriangularis,Mean_ctx_lh_pericalcarine,Mean_ctx_lh_postcentral,Mean_ctx_lh_posteriorcingulate,Mean_ctx_lh_precentral,Mean_ctx_lh_precuneus,Mean_ctx_lh_rostralanteriorcingulate,Mean_ctx_lh_rostralmiddlefrontal,Mean_ctx_lh_superiorfrontal,Mean_ctx_lh_superiorparietal,Mean_ctx_lh_superiortemporal,Mean_ctx_lh_supramarginal,Mean_ctx_lh_temporalpole,Mean_ctx_lh_transversetemporal,Mean_ctx_rh_bankssts,Mean_ctx_rh_caudalanteriorcingulate,Mean_ctx_rh_caudalmiddlefrontal,Mean_ctx_rh_cuneus,Mean_ctx_rh_entorhinal,Mean_ctx_rh_frontalpole,Mean_ctx_rh_fusiform,Mean_ctx_rh_inferiorparietal,Mean_ctx_rh_inferiortemporal,Mean_ctx_rh_insula,Mean_ctx_rh_isthmuscingulate,Mean_ctx_rh_lateraloccipital,Mean_ctx_rh_lateralorbitofrontal,Mean_ctx_rh_lingual,Mean_ctx_rh_medialorbitofrontal,Mean_ctx_rh_middletemporal,Mean_ctx_rh_paracentral,Mean_ctx_rh_parahippocampal,Mean_ctx_rh_parsopercularis,Mean_ctx_rh_parsorbitalis,Mean_ctx_rh_parstriangularis,Mean_ctx_rh_pericalcarine,Mean_ctx_rh_postcentral,Mean_ctx_rh_posteriorcingulate,Mean_ctx_rh_precentral,Mean_ctx_rh_precuneus,Mean_ctx_rh_rostralanteriorcingulate,Mean_ctx_rh_rostralmiddlefrontal,Mean_ctx_rh_superiorfrontal,Mean_ctx_rh_superiorparietal,Mean_ctx_rh_superiortemporal,Mean_ctx_rh_supramarginal,Mean_ctx_rh_temporalpole,Mean_ctx_rh_transversetemporal,bi_Accumbens_area,bi_Amygdala,bi_Caudate,bi_Cerebellum_Cortex,bi_Hippocampus,bi_Pallidum,bi_Putamen,bi_Thalamus,bi_VentralDC,bi_bankssts,bi_caudalanteriorcingulate,bi_caudalmiddlefrontal,bi_cuneus,bi_entorhinal,bi_frontalpole,bi_fusiform,bi_inferiorparietal,bi_inferiortemporal,bi_insula,bi_isthmuscingulate,bi_lateraloccipital,bi_lateralorbitofrontal,bi_lingual,bi_medialorbitofrontal,bi_middletemporal,bi_paracentral,bi_parahippocampal,bi_parsopercularis,bi_parsorbitalis,bi_parstriangularis,bi_pericalcarine,bi_postcentral,bi_posteriorcingulate,bi_precentral,bi_precuneus,bi_rostralanteriorcingulate,bi_rostralmiddlefrontal,bi_superiorfrontal,bi_superiorparietal,bi_superiortemporal,bi_supramarginal,bi_temporalpole,bi_transversetemporal
0,B10423472,Florbetapir,80.8687,0.911925,0.857168,0.733042,0.748900,0.663917,0.866686,1.199217,1.387516,0.993719,0.992500,1.264309,1.447619,1.336528,1.124297,1.159502,1.276200,1.280420,1.080324,1.007514,1.228982,1.439835,1.345137,1.167520,1.155619,1.251396,1.042559,1.045382,0.924735,1.435390,0.982222,1.200070,1.045288,1.266372,1.112922,1.122797,0.899106,1.208042,1.011490,1.167323,1.180958,1.045682,1.229686,1.097007,1.057292,1.085904,0.960775,0.860891,1.096538,0.946089,1.088230,1.173869,1.072606,1.051431,0.911522,1.063444,0.978218,1.123097,0.953441,1.195325,1.076235,1.022040,0.892560,1.354656,0.941203,1.175744,1.000528,1.182646,1.112481,1.153190,0.891791,1.253431,1.034269,1.166226,1.067204,1.017820,1.224819,1.116251,1.204253,1.211765,0.988561,0.901872,1.095075,0.964479,1.070215,1.167229,1.094925,1.041434,0.896564,1.050127,0.996157,1.080296,0.943707,1.252307,1.327945,1.035583,1,1.245483,

In [241]:
#save the a4 one with just tau suvr and no vent 
merged_a4_at_amypos.to_csv('./a4/merged_a4_at_amypos_novol.csv')
# save the adni one with just tau suvr and no vent 
merged_adni_at_amypos.to_csv('./adni/merged_adni_at_amy_pos_novol.csv')

#### now drop the left and right duplicates 
- also with left right from a4 stanford doumentation "We additionally created bilateral regions taking a volume weighted average across hemispheres. These are provided for in the bi_* columns" so maybe just use these
- in adni data it's going to be "RIGHT_**" OR "LEFT_*" OR "CTX_RH" OR "CTX_LH" 
- for adni some regions like orbiotfrontal gyrus, there are 3: orbiotfrontal gyrus, LEFT_ORBITOFRONTAL_GYRUS, and RIGHT_ etc so just take the one without left and right 


In [242]:
#find the adni brain regions list without "RIGHT_**" OR "LEFT_*" OR "CTX_RH" OR "CTX_LH" 
prefixes_to_remove = ['RIGHT_', 'LEFT_', 'CTX_RH', 'CTX_LH']
for prefix in prefixes_to_remove:
    adni_drop_cols = [col for col in merged_adni_at_amypos.columns if col.startswith(prefix)]
    merged_adni_at_amypos = merged_adni_at_amypos.drop(columns=adni_drop_cols)
#this leaves 47 cols 

In [243]:
merged_adni_at_amypos_bi = merged_adni_at_amypos
merged_adni_at_amypos_bi.to_csv("/home/mlauber/mci_mri_graph/data_paths_and_cleaning/data/intermediate_data/adni/merged_adni_at_amy_pos_bi.csv")
print(merged_adni_at_amypos_bi)

      RID TRACER_x  CENTILOIDS   LONIUID  SITEID        PTID    SCANDATE  \
0      31      FBP        97.0  I1596172      23  023-S-0031  2019-04-23   
1     112      FBP        89.0  I1600753     127  127-S-0112  2018-08-13   
2     120      FBP        32.0  I1596585      27  027-S-0120  2017-09-13   
3     210      FBP        54.0  I1599039      68  068-S-0210  2018-03-13   
4     377      FBP        85.0  I1598072      37  037-S-0377  2019-06-05   
5     416      FBP        62.0  I1600304     114  114-S-0416  2019-09-24   
6     467      FBP        78.0  I1598075      37  037-S-0467  2017-10-17   
7     555      FBP       114.0  I1597391      35  035-S-0555  2020-02-06   
8     618      FBP        46.0  I1597049      31  031-S-0618  2019-10-15   
9     626      FBP       123.0  I1595829      21  021-S-0626  2017-04-12   
10    677      FBP        45.0  I1597144      32  032-S-0677  2019-09-17   
11    679      FBP        54.0  I1614330      41  041-S-0679  2022-03-29   
12    731   

In [244]:
#drop a4 columns that have the string "lh" "rh" or "Right" or "Left"
pattern = r'(_lh_|_rh_|Left|Right)'
lr_cols = merged_a4_at_amypos.filter(regex=pattern, axis=1)
merged_a4_at_amypos_bi = merged_a4_at_amypos.drop(columns=lr_cols)
print(merged_a4_at_amypos_bi)
#this is dropping entorhinal becaise it is finidhing the rh pattern


           BID       ligand  centiloid  Mean_Brain_Stem  Mean_CC_Anterior  \
0    B10423472  Florbetapir    80.8687         0.911925          0.857168   
1    B10693248  Florbetapir   113.8210         1.000662          0.908223   
2    B10709603  Florbetapir    40.5933         1.007361          0.853996   
3    B10904689  Florbetapir   126.6360         0.920387          0.971428   
4    B11089276  Florbetapir    35.1012         1.035534          0.875824   
5    B11217281  Florbetapir   135.7900         1.010265          1.026229   
6    B11649219  Florbetapir    77.2073         0.927911          0.716755   
7    B12102791  Florbetapir    71.7152         1.004307          0.788493   
8    B12755986  Florbetapir    33.2705         0.914272          0.868799   
9    B12974065  Florbetapir    79.0380         0.941694          0.823991   
10   B13066573  Florbetapir    71.7152         0.948856          0.878936   
11   B13376106  Florbetapir    62.5617         0.988318          0.953917   

In [245]:
merged_a4_at_amypos_bi.to_csv("/home/mlauber/mci_mri_graph/data_paths_and_cleaning/data/intermediate_data/a4/merged_a4_at_amy_pos_bi.csv")

#### make all the a4 col names uppercase 

In [246]:
merged_a4_at_amypos_bi.columns = [col.upper() for col in merged_a4_at_amypos_bi.columns]
print(merged_a4_at_amypos_bi.columns)

Index(['BID', 'LIGAND', 'CENTILOID', 'MEAN_BRAIN_STEM', 'MEAN_CC_ANTERIOR',
       'MEAN_CC_CENTRAL', 'MEAN_CC_MID_ANTERIOR', 'MEAN_CC_MID_POSTERIOR',
       'MEAN_CC_POSTERIOR', 'BI_ACCUMBENS_AREA', 'BI_AMYGDALA', 'BI_CAUDATE',
       'BI_CEREBELLUM_CORTEX', 'BI_HIPPOCAMPUS', 'BI_PALLIDUM', 'BI_PUTAMEN',
       'BI_THALAMUS', 'BI_VENTRALDC', 'BI_BANKSSTS',
       'BI_CAUDALANTERIORCINGULATE', 'BI_CAUDALMIDDLEFRONTAL', 'BI_CUNEUS',
       'BI_ENTORHINAL', 'BI_FRONTALPOLE', 'BI_FUSIFORM', 'BI_INFERIORPARIETAL',
       'BI_INFERIORTEMPORAL', 'BI_INSULA', 'BI_ISTHMUSCINGULATE',
       'BI_LATERALOCCIPITAL', 'BI_LATERALORBITOFRONTAL', 'BI_LINGUAL',
       'BI_MEDIALORBITOFRONTAL', 'BI_MIDDLETEMPORAL', 'BI_PARACENTRAL',
       'BI_PARAHIPPOCAMPAL', 'BI_PARSOPERCULARIS', 'BI_PARSORBITALIS',
       'BI_PARSTRIANGULARIS', 'BI_PERICALCARINE', 'BI_POSTCENTRAL',
       'BI_POSTERIORCINGULATE', 'BI_PRECENTRAL', 'BI_PRECUNEUS',
       'BI_ROSTRALANTERIORCINGULATE', 'BI_ROSTRALMIDDLEFRONTAL',
      

In [247]:
#drop the mean columns from a4
a4_mean_drop_cols = [col for col in merged_a4_at_amypos_bi.columns if col.startswith('MEAN')]
merged_a4_at_amypos_bi = merged_a4_at_amypos_bi.drop(columns=a4_mean_drop_cols)

In [248]:
#remove the BI prefix from A4 
merged_a4_at_amypos_bi.columns = merged_a4_at_amypos_bi.columns.str.replace("BI_",'')

In [249]:
print(merged_a4_at_amypos_bi.columns)

Index(['BID', 'LIGAND', 'CENTILOID', 'ACCUMBENS_AREA', 'AMYGDALA', 'CAUDATE',
       'CEREBELLUM_CORTEX', 'HIPPOCAMPUS', 'PALLIDUM', 'PUTAMEN', 'THALAMUS',
       'VENTRALDC', 'BANKSSTS', 'CAUDALANTERIORCINGULATE',
       'CAUDALMIDDLEFRONTAL', 'CUNEUS', 'ENTORHINAL', 'FRONTALPOLE',
       'FUSIFORM', 'INFERIORPARIETAL', 'INFERIORTEMPORAL', 'INSULA',
       'ISTHMUSCINGULATE', 'LATERALOCCIPITAL', 'LATERALORBITOFRONTAL',
       'LINGUAL', 'MEDIALORBITOFRONTAL', 'MIDDLETEMPORAL', 'PARACENTRAL',
       'PARAHIPPOCAMPAL', 'PARSOPERCULARIS', 'PARSORBITALIS',
       'PARSTRIANGULARIS', 'PERICALCARINE', 'POSTCENTRAL',
       'POSTERIORCINGULATE', 'PRECENTRAL', 'PRECUNEUS',
       'ROSTRALANTERIORCINGULATE', 'ROSTRALMIDDLEFRONTAL', 'SUPERIORFRONTAL',
       'SUPERIORPARIETAL', 'SUPERIORTEMPORAL', 'SUPRAMARGINAL', 'TEMPORALPOLE',
       'TRANSVERSETEMPORAL'],
      dtype='object')


In [250]:
# merged_adni_at_amypos_bi = merged_adni_at_amypos_bi.drop(columns=['EXAMDATE','VISCODE','VISCODE2','update_stamp'])

In [251]:
#drop CTX prefix and drop SUVR suffix
merged_adni_at_amypos_bi.columns = merged_adni_at_amypos_bi.columns.str.replace("CTX_",'').str.replace('_SUVR','')

In [252]:
print(merged_adni_at_amypos_bi.columns) #in adni it's thalamus_proper so need to drop the proper for it to match 

Index(['RID', 'TRACER_x', 'CENTILOIDS', 'LONIUID', 'SITEID', 'PTID',
       'SCANDATE', 'PROCESSDATE', 'TRACER_y', 'META_TEMPORAL', 'ENTORHINAL',
       'INFERIORCEREBELLUM', 'CEREBRAL_WHITE_MATTER', 'BRAINSTEM',
       'CC_ANTERIOR', 'CC_CENTRAL', 'CC_MID_ANTERIOR', 'CC_MID_POSTERIOR',
       'CC_POSTERIOR', 'CSF', 'VENTRICLE_3RD', 'VENTRICLE_4TH', 'BANKSSTS',
       'CAUDALANTERIORCINGULATE', 'CAUDALMIDDLEFRONTAL', 'CUNEUS',
       'FRONTALPOLE', 'FUSIFORM', 'INFERIORPARIETAL', 'INFERIORTEMPORAL',
       'INSULA', 'ISTHMUSCINGULATE', 'LATERALOCCIPITAL',
       'LATERALORBITOFRONTAL', 'LINGUAL', 'MEDIALORBITOFRONTAL',
       'MIDDLETEMPORAL', 'PARACENTRAL', 'PARAHIPPOCAMPAL', 'PARSOPERCULARIS',
       'PARSORBITALIS', 'PARSTRIANGULARIS', 'PERICALCARINE', 'POSTCENTRAL',
       'POSTERIORCINGULATE', 'PRECENTRAL', 'PRECUNEUS',
       'ROSTRALANTERIORCINGULATE', 'ROSTRALMIDDLEFRONTAL', 'SUPERIORFRONTAL',
       'SUPERIORPARIETAL', 'SUPERIORTEMPORAL', 'SUPRAMARGINAL', 'TEMPORALPOLE',
     

In [253]:
merged_adni_at_amypos_bi= merged_adni_at_amypos_bi.rename(columns={'THALAMUS_PROPER': 'THALAMUS'})

In [254]:
print(merged_a4_at_amypos_bi.columns)

Index(['BID', 'LIGAND', 'CENTILOID', 'ACCUMBENS_AREA', 'AMYGDALA', 'CAUDATE',
       'CEREBELLUM_CORTEX', 'HIPPOCAMPUS', 'PALLIDUM', 'PUTAMEN', 'THALAMUS',
       'VENTRALDC', 'BANKSSTS', 'CAUDALANTERIORCINGULATE',
       'CAUDALMIDDLEFRONTAL', 'CUNEUS', 'ENTORHINAL', 'FRONTALPOLE',
       'FUSIFORM', 'INFERIORPARIETAL', 'INFERIORTEMPORAL', 'INSULA',
       'ISTHMUSCINGULATE', 'LATERALOCCIPITAL', 'LATERALORBITOFRONTAL',
       'LINGUAL', 'MEDIALORBITOFRONTAL', 'MIDDLETEMPORAL', 'PARACENTRAL',
       'PARAHIPPOCAMPAL', 'PARSOPERCULARIS', 'PARSORBITALIS',
       'PARSTRIANGULARIS', 'PERICALCARINE', 'POSTCENTRAL',
       'POSTERIORCINGULATE', 'PRECENTRAL', 'PRECUNEUS',
       'ROSTRALANTERIORCINGULATE', 'ROSTRALMIDDLEFRONTAL', 'SUPERIORFRONTAL',
       'SUPERIORPARIETAL', 'SUPERIORTEMPORAL', 'SUPRAMARGINAL', 'TEMPORALPOLE',
       'TRANSVERSETEMPORAL'],
      dtype='object')


#### now find over lapping regions (with same name)

In [255]:
#rename BID to rid in a4 to make it consistent with adni 
merged_a4_at_amypos_bi = merged_a4_at_amypos_bi.rename(columns={'BID':'RID'})
#rename ligand col to tracer 
merged_a4_at_amypos_bi = merged_a4_at_amypos_bi.rename(columns={'LIGAND':'TRACER'})
merged_a4_at_amypos_bi = merged_a4_at_amypos_bi.rename(columns={'CENTILOID':'CENTILOIDS'})


In [256]:
#get a list of regions in both dataframes (44 common regions)
common_regions = list(set(merged_adni_at_amypos_bi.columns)& set(merged_a4_at_amypos_bi.columns))
print(len(common_regions))
#now filter dfs based on common regions 
merged_a4_at_amypos_bi_harm = merged_a4_at_amypos_bi[common_regions]
merged_adni_at_amypos_bi_harm = merged_adni_at_amypos_bi[common_regions]
merged_a4_at_amypos_bi_harm = merged_a4_at_amypos_bi_harm.set_index('RID')
merged_adni_at_amypos_bi_harm = merged_adni_at_amypos_bi_harm.set_index('RID')


45


In [257]:
#round to 3 decimal points so its like the adni 
merged_a4_at_amypos_bi_harm = merged_a4_at_amypos_bi_harm.round(3)
print(merged_a4_at_amypos_bi_harm)

           SUPRAMARGINAL  AMYGDALA  MIDDLETEMPORAL  PARSOPERCULARIS  \
RID                                                                   
B10423472          0.987     1.328           1.118            1.106   
B10693248          1.146     1.660           1.298            1.140   
B10709603          1.036     1.276           1.118            1.064   
B10904689          1.142     1.436           1.198            1.081   
B11089276          1.098     1.238           1.161            1.175   
B11217281          1.232     1.326           1.088            1.035   
B11649219          1.044     1.078           1.132            1.039   
B12102791          1.141     1.539           1.464            1.167   
B12755986          1.037     1.236           1.122            1.083   
B12974065          1.108     1.149           1.213            1.127   
B13066573          1.125     1.210           1.186            1.074   
B13376106          1.020     1.210           1.073            1.038   
B14055

In [258]:
print(merged_adni_at_amypos_bi_harm)

      SUPRAMARGINAL  AMYGDALA  MIDDLETEMPORAL  PARSOPERCULARIS  \
RID                                                              
31            1.259     1.479           1.256            1.158   
112           1.538     2.350           1.690            1.687   
120           1.205     1.213           1.241            1.088   
210           1.513     1.134           1.475            1.381   
377           1.603     1.515           1.767            1.456   
416           1.292     1.104           1.325            1.235   
467           1.816     2.348           2.161            1.898   
555           1.231     1.430           1.369            1.175   
618           1.144     1.506           1.281            1.031   
626           1.455     1.604           1.960            1.307   
677           1.752     1.228           1.483            1.668   
679           1.106     1.146           1.119            1.074   
731           1.214     1.251           1.107            1.302   
800       

In [259]:
#save the new csvs that have the same regions across both 
merged_a4_at_amypos_bi_harm.to_csv('/home/mlauber/mci_mri_graph/data_paths_and_cleaning/data/intermediate_data/a4/merged_a4_at_amy_pos_bi_harm.csv')
merged_adni_at_amypos_bi_harm.to_csv('/home/mlauber/mci_mri_graph/data_paths_and_cleaning/data/intermediate_data/adni/merged_adni_at_amy_pos_bi_harm.csv')

check they both have the same number of regions 

In [260]:
print(merged_a4_at_amypos_bi_harm.shape)
print(merged_adni_at_amypos_bi_harm.shape)

(373, 44)
(398, 44)
